In [29]:
import pandas as pd
import glob

In [5]:
pd.read_csv("../csv/result_07_31_from_07_29.csv",index_col=0).drop(["0"],axis=1)

,xcoord,ycoord
0,778875,3995835
1,778885,3995835
2,778895,3995835
3,778905,3995835
4,778915,3995835
...,...,...
129138,777475,3991515
129139,777485,3991515
129140,777495,3991515
129141,777475,3991505


In [10]:
pd.read_csv("../data/SV/Train_2021_07_21_SV_1_10m.csv")

,fid,VALUE,SAMPLE_1,SAMPLE_2,SAMPLE_3,SAMPLE_4
0,1,1140,194,318,303,63
1,2,1140,203,319,292,59
2,3,1135,190,329,289,62
3,4,1137,197,330,305,66
4,5,1126,207,334,314,82
...,...,...,...,...,...,...
493,494,1221,275,367,311,112
494,495,1255,250,348,296,152
495,496,1244,268,348,297,145
496,497,1208,266,358,317,129


In [34]:
data_paths_10m = glob.glob("../data/Suwako_all/2021_0[7-9]_*_10m.csv")
data_paths_20m = glob.glob("../data/Suwako_all/2021_0[7-9]_*_20m.csv")

In [35]:
data_paths_10m

['../data/Suwako_all\\2021_07_21_all_10m.csv',
 '../data/Suwako_all\\2021_08_05_all_10m.csv',
 '../data/Suwako_all\\2021_08_30_all_10m.csv',
 '../data/Suwako_all\\2021_09_19_all_10m.csv',
 '../data/Suwako_all\\2021_09_27_all_10m.csv',
 '../data/Suwako_all\\2021_09_29_all_10m.csv']

In [36]:
data_paths_20m

['../data/Suwako_all\\2021_07_21_all_20m.csv',
 '../data/Suwako_all\\2021_08_05_all_20m.csv',
 '../data/Suwako_all\\2021_08_30_all_20m.csv',
 '../data/Suwako_all\\2021_09_19_all_20m.csv',
 '../data/Suwako_all\\2021_09_27_all_20m.csv',
 '../data/Suwako_all\\2021_09_29_all_20m.csv']

In [39]:
month_list = [7, 8, 8, 9, 9, 9]

In [47]:
df = pd.DataFrame()

for i in range(len(data_paths_10m)):
    df_10m = pd.read_csv(data_paths_10m[i]).drop(["fid", "VALUE"], axis=1)
    df_10m.columns = ["xcoord", "ycoord", "Blue", "Green", "Red", "NIR"]
    
    df_20m = pd.read_csv(data_paths_20m[i]).drop(["fid", "VALUE", "SAMPLE_8", "SAMPLE_9", "SAMPLE_10", "SAMPLE_11"], axis=1)
    df_20m.columns = ["xcoord", "ycoord", "B5", "B6", "B7", "B8A", "B11", "B12", "CLD"]

    df_10m20m = pd.merge(df_10m, df_20m, on=['xcoord', 'ycoord'])

    df_10m20m["month"] = month_list[i]

    df = pd.concat([df, df_10m20m])
    df = df.reset_index(drop=True)

df["sf1_ave"]     = (df["NIR"] - ((df["Blue"] + df["Green"] + df["Red"])/3))/(df["NIR"] + ((df["Blue"] + df["Green"] + df["Red"])/3))
df["WAVI"]        = (1.5*(df["NIR"] - df["Blue"]))/(0.5 + df["NIR"] + df["Blue"])
df["NDAVI"]       = (df["NIR"] - df["Blue"])/(df["NIR"] + df["Blue"])
df["NDVI"]        = (df["NIR"] - df["Red"])/(df["NIR"] + df["Red"])
df["ave_123"]     = ((df["Blue"] + df["Green"] + df["Red"])/3)
df["MNDWI"]       = (df["Green"] - df["B11"])/(df["Green"] + df["B11"])
df["NDMI"]        = (df["NIR"] - df["B11"])/(df["NIR"] + df["B11"])
df["NDCI"]        = (df["B5"] - df["Red"])/(df["Red"] + df["B5"])
df["TCT_Wetness"] = 0.2578*df["Blue"] + 0.2305*df["Green"] + 0.0883*df["Red"] + 0.1071*df["NIR"] + (-0.7611)*df["B11"] + (-0.5308)*df["B12"]
df["FAI"]         = df["NIR"] - (df["Red"] + (df["B11"] - df["Red"]) * ((842 - 665) / (1610 - 665)))

df = df[df["CLD"]>30]
df

,xcoord,ycoord,Blue,Green,Red,NIR,B5,B6,B7,B8A,...,sf1_ave,WAVI,NDAVI,NDVI,ave_123,MNDWI,NDMI,NDCI,TCT_Wetness,FAI
0,238491.434662,3.995299e+06,507.0,836.0,503.0,3548.0,1048.0,2745.0,2995.0,3173.0,...,0.704404,1.124769,0.749938,0.751666,615.333333,-0.366906,0.325612,0.351386,-1162.6160,2801.133333
1,238501.414486,3.995299e+06,457.0,745.0,465.0,2574.0,1017.0,2143.0,2459.0,2579.0,...,0.644904,1.047501,0.698449,0.693978,555.666667,-0.331539,0.268605,0.372470,-974.3804,1918.139683
2,238511.394308,3.995298e+06,410.0,733.0,440.0,2450.0,1017.0,2143.0,2459.0,2579.0,...,0.645584,1.069743,0.713287,0.695502,527.666667,-0.338746,0.245552,0.396019,-1004.7509,1814.457143
3,238521.374129,3.995297e+06,389.0,651.0,436.0,2018.0,1031.0,2298.0,2336.0,2526.0,...,0.607968,1.014953,0.676776,0.644662,492.000000,-0.324338,0.225258,0.405590,-847.8425,1424.666667
4,238531.353949,3.995297e+06,411.0,700.0,462.0,2124.0,1031.0,2298.0,2336.0,2526.0,...,0.604028,1.013410,0.675740,0.642691,524.333333,-0.291498,0.249412,0.381112,-817.2280,1509.536508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774853,236828.064106,3.991074e+06,469.0,655.0,418.0,838.0,966.0,996.0,1088.0,1160.0,...,0.239645,0.423327,0.282326,0.334395,514.000000,-0.153747,-0.031774,0.395954,-599.5974,331.031746
774854,236838.044168,3.991073e+06,466.0,580.0,388.0,218.0,1098.0,1252.0,1216.0,1161.0,...,-0.373563,-0.543462,-0.362573,-0.280528,478.000000,-0.323221,-0.677515,0.477793,-964.0860,-309.726984
774855,236848.024229,3.991073e+06,646.0,651.0,509.0,604.0,1098.0,1252.0,1216.0,1161.0,...,0.001658,-0.050380,-0.033600,0.085355,602.000000,-0.270588,-0.304948,0.366521,-849.2916,-22.063492
774856,236827.448231,3.991064e+06,655.0,785.0,544.0,1926.0,966.0,996.0,1088.0,1160.0,...,0.488792,0.738524,0.492445,0.559514,661.333333,-0.064362,0.366442,0.279470,-394.0310,1316.631746


In [19]:
df_10m = pd.read_csv("../data/Suwako_all/2021_06_01_all_10m.csv").drop(["fid", "VALUE"], axis=1)
df_10m.columns = ["xcoord", "ycoord", "Blue", "Green", "Red", "NIR"]
df_10m

,xcoord,ycoord,Blue,Green,Red,NIR
0,238491.434662,3.995299e+06,507,738,432,3068
1,238501.414486,3.995299e+06,430,610,406,2100
2,238511.394308,3.995298e+06,384,508,379,1638
3,238521.374129,3.995297e+06,337,487,324,1182
4,238531.353949,3.995297e+06,311,480,312,1060
...,...,...,...,...,...,...
129138,236828.064106,3.991074e+06,330,473,328,556
129139,236838.044168,3.991073e+06,287,432,340,343
129140,236848.024229,3.991073e+06,387,417,387,397
129141,236827.448231,3.991064e+06,727,750,623,1664


In [24]:
df_20m = pd.read_csv("../data/Suwako_all/2021_06_01_all_20m.csv").drop(["fid", "VALUE", "SAMPLE_8", "SAMPLE_9", "SAMPLE_10", "SAMPLE_11"], axis=1)
df_20m.columns = ["xcoord", "ycoord", "B5", "B6", "B7", "B8A", "B11", "B12", "CLD"]
df_20m

,xcoord,ycoord,B5,B6,B7,B8A,B11,B12,CLD
0,238491.434662,3.995299e+06,951.0,2082.0,2588.0,2566.0,1514.0,895.0,102.0
1,238501.414486,3.995299e+06,738.0,1529.0,1632.0,1703.0,1079.0,660.0,102.0
2,238511.394308,3.995298e+06,738.0,1529.0,1632.0,1703.0,1079.0,660.0,102.0
3,238521.374129,3.995297e+06,663.0,1344.0,1302.0,1527.0,932.0,490.0,102.0
4,238531.353949,3.995297e+06,663.0,1344.0,1302.0,1527.0,932.0,490.0,102.0
...,...,...,...,...,...,...,...,...,...
129138,236828.064106,3.991074e+06,760.0,921.0,1212.0,1083.0,826.0,583.0,102.0
129139,236838.044168,3.991073e+06,951.0,1026.0,788.0,884.0,935.0,642.0,102.0
129140,236848.024229,3.991073e+06,951.0,1026.0,788.0,884.0,935.0,642.0,102.0
129141,236827.448231,3.991064e+06,760.0,921.0,1212.0,1083.0,826.0,583.0,102.0


In [26]:
df_10m20m = pd.merge(df_10m, df_20m, on=['xcoord', 'ycoord'])
df_10m20m

,xcoord,ycoord,Blue,Green,Red,NIR,B5,B6,B7,B8A,B11,B12,CLD
0,238491.434662,3.995299e+06,507,738,432,3068,951.0,2082.0,2588.0,2566.0,1514.0,895.0,102.0
1,238501.414486,3.995299e+06,430,610,406,2100,738.0,1529.0,1632.0,1703.0,1079.0,660.0,102.0
2,238511.394308,3.995298e+06,384,508,379,1638,738.0,1529.0,1632.0,1703.0,1079.0,660.0,102.0
3,238521.374129,3.995297e+06,337,487,324,1182,663.0,1344.0,1302.0,1527.0,932.0,490.0,102.0
4,238531.353949,3.995297e+06,311,480,312,1060,663.0,1344.0,1302.0,1527.0,932.0,490.0,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129138,236828.064106,3.991074e+06,330,473,328,556,760.0,921.0,1212.0,1083.0,826.0,583.0,102.0
129139,236838.044168,3.991073e+06,287,432,340,343,951.0,1026.0,788.0,884.0,935.0,642.0,102.0
129140,236848.024229,3.991073e+06,387,417,387,397,951.0,1026.0,788.0,884.0,935.0,642.0,102.0
129141,236827.448231,3.991064e+06,727,750,623,1664,760.0,921.0,1212.0,1083.0,826.0,583.0,102.0


In [28]:
df_10m20m["NDVI"] = (df_10m20m["NIR"] - df_10m20m["Red"]) / (df_10m20m["NIR"] + df_10m20m["Red"])
df_10m20m["NDWI"] = (df_10m20m["NIR"] - df_10m20m["B11"]) / (df_10m20m["NIR"] + df_10m20m["B11"])
df_10m20m

,xcoord,ycoord,Blue,Green,Red,NIR,B5,B6,B7,B8A,B11,B12,CLD,NDVI,NDWI
0,238491.434662,3.995299e+06,507,738,432,3068,951.0,2082.0,2588.0,2566.0,1514.0,895.0,102.0,0.753143,0.339153
1,238501.414486,3.995299e+06,430,610,406,2100,738.0,1529.0,1632.0,1703.0,1079.0,660.0,102.0,0.675978,0.321170
2,238511.394308,3.995298e+06,384,508,379,1638,738.0,1529.0,1632.0,1703.0,1079.0,660.0,102.0,0.624194,0.205742
3,238521.374129,3.995297e+06,337,487,324,1182,663.0,1344.0,1302.0,1527.0,932.0,490.0,102.0,0.569721,0.118259
4,238531.353949,3.995297e+06,311,480,312,1060,663.0,1344.0,1302.0,1527.0,932.0,490.0,102.0,0.545190,0.064257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129138,236828.064106,3.991074e+06,330,473,328,556,760.0,921.0,1212.0,1083.0,826.0,583.0,102.0,0.257919,-0.195369
129139,236838.044168,3.991073e+06,287,432,340,343,951.0,1026.0,788.0,884.0,935.0,642.0,102.0,0.004392,-0.463224
129140,236848.024229,3.991073e+06,387,417,387,397,951.0,1026.0,788.0,884.0,935.0,642.0,102.0,0.012755,-0.403904
129141,236827.448231,3.991064e+06,727,750,623,1664,760.0,921.0,1212.0,1083.0,826.0,583.0,102.0,0.455181,0.336546
